from google.colab import drive
drive.mount('/content/drive')

In [272]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import mean_squared_error,mean_absolute_error


In [222]:
Dataset_path = "./DataSets/"
Movies_metadata = pd.read_csv(Dataset_path+"MoviesInfo.csv")
ratings = pd.read_csv(Dataset_path+"ml-latest/ratings.csv")

In [223]:
Movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41896 entries, 0 to 41895
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movieId                41896 non-null  int64  
 1   genres                 41896 non-null  object 
 2   imdb_title_id          41896 non-null  object 
 3   title                  41896 non-null  object 
 4   original_title         41896 non-null  object 
 5   year                   41896 non-null  int64  
 6   date_published         41896 non-null  object 
 7   genre                  41896 non-null  object 
 8   duration               41896 non-null  int64  
 9   country                41890 non-null  object 
 10  language               41648 non-null  object 
 11  director               41886 non-null  object 
 12  writer                 41621 non-null  object 
 13  production_company     40944 non-null  object 
 14  actors                 41877 non-null  object 
 15  de

In [171]:
obj_Movies_metadata = Movies_metadata.select_dtypes(include=['object']).copy()
obj_Movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41896 entries, 0 to 41895
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   genres                 41896 non-null  object
 1   imdb_title_id          41896 non-null  object
 2   title                  41896 non-null  object
 3   original_title         41896 non-null  object
 4   date_published         41896 non-null  object
 5   genre                  41896 non-null  object
 6   country                41890 non-null  object
 7   language               41648 non-null  object
 8   director               41886 non-null  object
 9   writer                 41621 non-null  object
 10  production_company     40944 non-null  object
 11  actors                 41877 non-null  object
 12  description            41589 non-null  object
 13  budget                 15453 non-null  object
 14  usa_gross_income       13291 non-null  object
 15  worlwide_gross_inco

In [172]:
myobj = ['country','language','director','production_company']

for i in myobj:
    obj_Movies_metadata[i] = obj_Movies_metadata[i].astype('category')
    Movies_metadata[i] = obj_Movies_metadata[i].cat.codes


In [173]:
Movies_metadata = Movies_metadata.fillna(100000)
Movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41896 entries, 0 to 41895
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movieId                41896 non-null  int64  
 1   genres                 41896 non-null  object 
 2   imdb_title_id          41896 non-null  object 
 3   title                  41896 non-null  object 
 4   original_title         41896 non-null  object 
 5   year                   41896 non-null  int64  
 6   date_published         41896 non-null  object 
 7   genre                  41896 non-null  object 
 8   duration               41896 non-null  int64  
 9   country                41896 non-null  int16  
 10  language               41896 non-null  int16  
 11  director               41896 non-null  int16  
 12  writer                 41896 non-null  object 
 13  production_company     41896 non-null  int16  
 14  actors                 41896 non-null  object 
 15  de

In [174]:
data =  Movies_metadata[['year','duration','metascore','reviews_from_users','reviews_from_critics'
                              ,'country','language','director','production_company','mean_vote']]

In [175]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled = scaler.fit_transform(data.drop(['mean_vote'],axis=1))
data_norm = pd.DataFrame(scaled,columns=(data.drop(['mean_vote'],axis=1).columns))
data_norm= data_norm.join(data['mean_vote'])


from scipy import stats
z_scores = stats.zscore(data_norm.drop(['mean_vote'],axis=1))
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
newdata = data_norm[filtered_entries]

newdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38725 entries, 0 to 41894
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  38725 non-null  float64
 1   duration              38725 non-null  float64
 2   metascore             38725 non-null  float64
 3   reviews_from_users    38725 non-null  float64
 4   reviews_from_critics  38725 non-null  float64
 5   country               38725 non-null  float64
 6   language              38725 non-null  float64
 7   director              38725 non-null  float64
 8   production_company    38725 non-null  float64
 9   mean_vote             38725 non-null  float64
dtypes: float64(10)
memory usage: 3.2 MB


### Without Classing of target

In [176]:
target = newdata['mean_vote'] 
predictors = newdata.drop(['mean_vote'],axis=1)

In [177]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30, random_state=40)


In [257]:
model = Sequential()
model.add(Dense(900, activation='relu', input_dim=9))
model.add(Dense(300, activation='tanh'))
model.add(Dense(150, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_squared_error'])

In [258]:
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
2711/2711 [==============================] - 18s 6ms/step - loss: 0.7478 - mean_squared_error: 0.9986
Epoch 2/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.7033 - mean_squared_error: 0.8495
Epoch 3/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.6946 - mean_squared_error: 0.8325
Epoch 4/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.6855 - mean_squared_error: 0.8163
Epoch 5/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.6802 - mean_squared_error: 0.8071
Epoch 6/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.6780 - mean_squared_error: 0.8044
Epoch 7/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.6780 - mean_squared_error: 0.8043
Epoch 8/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.6716 - mean_squared_error: 0.7920
Epoch 9/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.6720 - mean_squar

In [ ]:
y_pred= model.predict(X_test)

In [259]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores[1], 1 - scores[1]))    

Accuracy on test data: 0.7781487703323364% 
 Error on test data: 0.22185122966766357


In [260]:
print('MSE:',mean_squared_error(y_test, y_pred))
print('MAE:',mean_absolute_error(y_test, y_pred))
print('RMSE:',np.sqrt(mean_squared_error(y_test, y_pred)))

MSE: 0.7781488182432842
MAE: 0.6611473182902152
RMSE: 0.882127438777008


### With Classing mean_vote

In [266]:
newdata['mean_vote_class'] = 0
newdata['mean_vote_class'][newdata['mean_vote']>3.5] = 1
newdata['mean_vote_class'][newdata['mean_vote']<=3.5] = 0


<ipython-input-266-31771ccae61c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['mean_vote_class'] = 0
<ipython-input-266-31771ccae61c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['mean_vote_class'][newdata['mean_vote']>3.5] = 1
C:\Users\PJ\anaconda3\lib\site-packages\pandas\core\generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_in

In [267]:
target = newdata['mean_vote_class'] 
predictors = newdata.drop(['mean_vote','mean_vote_class'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30, random_state=40)


In [275]:
model = Sequential()
model.add(Dense(900, activation='relu', input_dim=9))
model.add(Dense(300, activation='tanh'))
model.add(Dense(150, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_squared_error'])

In [276]:
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
2711/2711 [==============================] - 18s 6ms/step - loss: 0.0148 - mean_squared_error: 0.0148
Epoch 2/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0148 - mean_squared_error: 0.0148
Epoch 3/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0148 - mean_squared_error: 0.0148
Epoch 4/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0148 - mean_squared_error: 0.0148
Epoch 5/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0148 - mean_squared_error: 0.0148
Epoch 6/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0148 - mean_squared_error: 0.0148
Epoch 7/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0148 - mean_squared_error: 0.0148
Epoch 8/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0148 - mean_squared_error: 0.0148
Epoch 9/20
2711/2711 [==============================] - 17s 6ms/step - loss: 0.0148 - mean_squar

In [277]:
y_pred= model.predict(X_test)

In [278]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores[1], 1 - scores[1])) 

Accuracy on test data: 0.015062833204865456% 
 Error on test data: 0.9849371667951345


In [279]:
print('MSE:',mean_squared_error(y_test, y_pred))
print('MAE:',mean_absolute_error(y_test, y_pred))
print('RMSE:',np.sqrt(mean_squared_error(y_test, y_pred)))

MSE: 0.015062833534171113
MAE: 0.015062833534171113
RMSE: 0.12273073589843382
